In [2]:
from ultralytics import YOLO
import numpy as np
import cv2


In [3]:
# palmer is poorly trained model
model = YOLO("palmer.onnx", task='detect')

In [4]:
results = model.predict(source="test_data/drone_trim.mov", save=False)  # save plotted images

Loading palmer.onnx for ONNX Runtime inference...


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/206) C:\Users\Lenovo\repos\starleaf\test_data\drone_trim.mov: 640x640 300 sws, 34.4ms
video 1/1 (2/206) C:\Users\Lenovo\repos\starleaf\test_data\drone_trim.mov: 640x640 300 sws, 31.7ms
video 1/1 (3/206) C:\Users\Lenovo\repos\starleaf\test_data\drone_trim.mov: 640x640 300 sws, 31.3ms
video 1/1 (4/206) C:\Users\Lenovo\repos\starleaf\test_data\drone_trim.mov: 640x640 300 sws, 31.3ms
video 1/1 

In [5]:
# Generate manually signed image

orimages    = []
modimages   = []
boxes       = []
names       = []


for result in results:
    name = result.names
    orig = result.orig_img
    box = result.boxes
    modimage = orig.copy()
    
    names.append(name)
    orimages.append(orig)
    boxes.append(box)

    for cbox in box.xywh:
        x, y, w, h = cbox 
        cx = int(x+w/2)
        cy = int(y+h/2)
        ctr = (cx, cy)
        col = (50, 0, 255) 
        rad = int((w+h)/8)
        thic = 3
        cv2.circle(modimage, ctr, rad, col, thic)

    count_text = f"count: {len(box)}"
    cv2.putText(modimage, count_text, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (50, 0, 255), 4, cv2.LINE_8)
    modimages.append(modimage)




In [6]:
# create result as movies

img = modimages[0]
height, width, _ = img.shape

# Create a video writer object
video = cv2.VideoWriter("result.avi", cv2.VideoWriter_fourcc(*"MJPG"), 10, (width, height))

# Loop over the images and write them to the video
for image in modimages:
    video.write(image)

# Release the video writer
video.release()

In [90]:
# better write to a text file

counts = [len(box) for box in boxes]
import pandas as pd

df = pd.DataFrame({
    "tree_count" : counts
})

In [1]:
df.to_csv("count_history.csv")

NameError: name 'df' is not defined